![title](https://user-images.githubusercontent.com/11041248/39694897-20056604-51e9-11e8-9bb4-5ac921c52e4b.png)

![title](https://user-images.githubusercontent.com/11041248/39694898-2032cab8-51e9-11e8-956e-1720936eae15.png)

![title](https://user-images.githubusercontent.com/11041248/39694899-20685b74-51e9-11e8-9b27-8439de2b4856.png)

![title](https://user-images.githubusercontent.com/11041248/39694900-209b8008-51e9-11e8-8c43-e3cdf0bdc1f5.png)

![title](https://user-images.githubusercontent.com/11041248/39694901-20c9c0bc-51e9-11e8-9f6d-652037aaf256.png)

![title](https://user-images.githubusercontent.com/11041248/39694902-20f85dfa-51e9-11e8-84a2-72e37734328e.png)

![title](https://user-images.githubusercontent.com/11041248/39694904-2159590c-51e9-11e8-8ed9-cf340005f496.png)

![title](https://user-images.githubusercontent.com/11041248/39694905-218045b2-51e9-11e8-9ca8-b1bb40362eeb.png)

![title](https://user-images.githubusercontent.com/11041248/39694906-21a7b502-51e9-11e8-87e9-1b15de2b84cd.png)

![title](https://user-images.githubusercontent.com/11041248/39694907-21d5b1d2-51e9-11e8-99f1-44b22d570bac.png)

![title](https://user-images.githubusercontent.com/11041248/39694908-21fc7cfe-51e9-11e8-9721-8e658fea77a4.png)

![title](https://user-images.githubusercontent.com/11041248/39694911-224183e4-51e9-11e8-8289-aec684bf35dc.png)

![title](https://user-images.githubusercontent.com/11041248/39694913-2268aac8-51e9-11e8-8799-ddfb1b38faa7.png)

![title](https://user-images.githubusercontent.com/11041248/39694914-228f3b84-51e9-11e8-8cfb-29ec95df0547.png)

In [ ]:
# Importing our libraries

import geopyspark as gps
import numpy as np
import pyproj
import fiona
import folium

from functools import partial
from shapely.geometry import Point
from shapely.ops import transform
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession

In [ ]:
# Setting up the SparkContext

conf = gps.geopyspark_conf(appName="geopython-notebook-emr", master='local[*]')
conf.set('spark.default.parallelism', 8)
conf.set('spark.ui.enabled', True)
conf.set('spark.yarn.executor.memoryOverhead', '1G')
conf.set('spark.yarn.driver.memoryOverhead', '1G')
conf.set('spark.master.memory', '9500M')
conf.set('spark.dynamicAllocation.enabled', True)
conf.set('spark.shuffle.service.enabled', True)
conf.set('spark.shuffle.compress', True)
conf.set('spark.shuffle.spill.compress', True)
conf.set('spark.rdd.compress', True)
conf.set('spark.driver.maxResultSize', '3G')
conf.set('spark.task.maxFailures', '33')
conf.set('spark.executor.extraJavaOptions', '-XX:+UseParallelGC')

sc = SparkContext(conf=conf)

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoopConf.set("fs.s3.awsAccessKeyId", '')
hadoopConf.set("fs.s3.awsSecretAccessKey", '')

pysc = gps.get_spark_context()
session = SparkSession.builder.config(conf=pysc.getConf()).enableHiveSupport().getOrCreate()

![title](https://user-images.githubusercontent.com/11041248/39694915-22b74ca0-51e9-11e8-9aac-7598c8587ccd.png)

![title](https://user-images.githubusercontent.com/11041248/39694916-22e14a14-51e9-11e8-80cb-7eb6048f2f3a.png)

![title](https://user-images.githubusercontent.com/11041248/39694917-2304a590-51e9-11e8-8944-abe11bbc8dfc.png)

In [ ]:
# produce_tiled_layer is a helper function made specifically for this presentation
source_layer = gps.produce_tiled_layer(value=1)

source_layer.to_numpy_rdd().values().collect()[0].cells

In [ ]:
source_layer_plus_one = source_layer + 1

source_layer_plus_one.to_numpy_rdd().values().collect()[0].cells

In [ ]:
source_layer_plus_source = source_layer + source_layer

source_layer_plus_source.to_numpy_rdd().values().collect()[0].cells

![title](https://user-images.githubusercontent.com/11041248/39694918-233413de-51e9-11e8-919f-c1f44c237e8c.png)

In [ ]:
# Creates a 3x3 neighborhood for the focal operatation
square = gps.Square(1)

# Performs a focal Sum operation using the neighborhood
summed_layer = source_layer.focal(operation=gps.Operation.SUM, neighborhood=square)

summed_layer.to_numpy_rdd().values().collect()[0].cells

![title](https://user-images.githubusercontent.com/11041248/39694919-2366f06a-51e9-11e8-88de-52143b41d4be.png)

![title](https://user-images.githubusercontent.com/11041248/39697266-a1d44c5c-51f0-11e8-9166-a32928f25b41.png)

# Creating a Friction Layer of the Continental United States

Friction Layer Factors:
 - Roads
 - Trails and Paths
 - Waterways
 - Land Cover
 - Elevation

## Establishing Shared Values

In [ ]:
# The URI that the layers will be saved/read from
layer_uri = "s3://geopyspark-demo/geopython/catalog/emr"

# The AttributeStore for the URI
store = gps.AttributeStore(layer_uri)

## Reading and Formatting the OSM Data

### Roads and Trails

In [ ]:
# Read in the OSM data as an ORC file
file_uri = "s3://geotrellis-test/xterrain/continental-us.orc"
osm_dataframe = session.read.orc(file_uri)

In [ ]:
# Get all of the lines that are contained within the DataFrame
osm = gps.vector_pipe.osm_reader.from_dataframe(osm_dataframe)
lines = osm.get_line_features_rdd()

In [ ]:
# Only roads/paths are of interest
highways = lines.filter(lambda feature: 'highway' in feature.properties.tags)

waterways = lines.filter(lambda feature: 'waterway' in feature.properties.tags)

In [ ]:
path_tags = ['footway', 'steps', 'bridleway', 'path', 'cycleway', 'escalator']

# Filter out the highways into roads and paths

roads = highways.filter(lambda feature: feature.properties.tags['highway'] not in path_tags)
paths = highways.filter(lambda feature: feature.properties.tags['highway'] in path_tags)

In [ ]:
# Encode the the paths with the default walking speed
path_features = paths\
    .map(lambda feature: gps.Feature(feature.geometry, gps.CellValue(3.74, 3.74)))\
    .persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
# This cell contains the logic that assigns each section of road a
# speed based on the type of road that section is.

default_speeds = {
    'motorway': 65,
    'trunk': 45,
    'primary': 40,
    'secondary': 35,
    'tertiary': 30,
    'unclassified': 20,
    'residential': 20,
    'service': 15,
    'motorway_link': 45,
    'trunk_link': 40,
    'primary_link': 35,
    'secondary_link': 30,
    'tertiary_link': 25,
    'living_street': 5,
    'pedestrian': 5,
    'track': 15,
    'road': 30}

words = ['maxspeed', 'ambiguous', 'signals', 
         'none', 'walk', 'variable', 
         'national', 'fixme', 'unposted', 'implicit']

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def default_speed(highway):
    if not highway in default_speeds:
        return default_speeds['road']
    else:
        return default_speeds[highway]

def get_maxspeed(speed, units, highway):
    speeds = speed.split(';|,-')
    maxspeed = 0
    for sp in speeds:
        sp = sp.replace(units, '')
        if (is_number(sp)):
            if units == 'kph':
                sp = float(sp) / 1.609344 
            elif units == 'knots':
                sp = 0.868976 * float(knots)
            else:
                sp = float(sp)
                
            if sp > maxspeed:
                maxspeed = sp
    if maxspeed > 0:
        speed = maxspeed
    else:
        speed = default_speed(highway)

    return speed

def get_highway_cellvalue(osm_feature):   
    highway = osm_feature.properties.tags['highway']
    speed = osm_feature.properties.tags.get('maxspeed', '')
                                
    speed = speed.lower().strip()
        
    # if we don't have a speed, give it a default
    if len(speed) == 0:
        speed = default_speed(highway)
    elif not is_number(speed):
        if 'kph' in speed:
            speed = get_maxspeed(speed, 'kph', highway)
        elif 'km/h' in speed:
            speed = get_maxspeed(speed, 'km/h', highway)
        elif 'kmh' in speed:
            speed = get_maxspeed(speed, 'kmh', highway)
        elif 'mph' in speed:
            speed = get_maxspeed(speed, 'mph', highway)
        elif 'knots' in speed:
            speed = get_maxspeed(speed, 'knots', highway)
        elif speed in words:
            speed = default_speed(highway)
        else:
            speed = get_maxspeed(speed, '', highway)            
    if float(speed) <= 0.0:
        speed = default_speed(highway)

    speed = float(speed)
    return gps.CellValue(speed, speed)

In [ ]:
# Encode the road speeds as feature properties for rasterization
road_features = roads\
    .map(lambda feature: gps.Feature(feature.geometry, get_highway_cellvalue(feature)))\
    .persist(StorageLevel.MEMORY_AND_DISK_SER)

### Waterways

In [ ]:
waterways_pmts_map = {
    'river': 0.3,
    'stream': 0.7,
    'brook': 0.8,
    'canal': 0.35,
    'drain': 0.85,
    'ditch': 0.8
}

def get_waterway_cellvalue(feature):
    waterway = feature.properties.tags['waterway']
    
    pmt = waterways_pmts_map.get(waterway)
    
    if pmt:
        value = pmt
    else:
        value = 0
        
    return gps.CellValue(value, value)

In [ ]:
waterways = lines.filter(lambda feature: 'waterway' in feature.properties.tags)

In [ ]:
# Encode the water speeds as feature properties for rasterization
waterway_features = waterways\
    .map(lambda feature: gps.Feature(feature.geometry, get_waterway_cellvalue(feature)))\
    .persist(StorageLevel.MEMORY_AND_DISK_SER)

## Rasterizing OSM Features

In [ ]:
# Combine the roads, paths, and waterways into one RDD and then rasterize them
osm_raster = gps.geotrellis.rasterize_features(
    features = pysc.union([road_features, path_features, waterway_features]),
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    partition_strategy = gps.SpatialPartitionStrategy(1000))\
.convert_data_type(gps.CellType.FLOAT32, -2147483648.0)\
.persist(StorageLevel.MEMORY_AND_DISK_SER)

## Saving the Rasterized OSM Features

In [ ]:
tiled_osm = osm_raster.tile_to_layout(gps.GlobalLayout(), target_crs=3857).with_no_data(0.0)
osm_pyramid = tiled_osm.pyramid(partition_strategy=gps.SpatialPartitionStrategy(1000))

osm_layer_name = "rasterized-osm-features"

# Save layer histogram for later use
osm_hist = osm_pyramid.get_histogram()
store.layer(osm_layer_name).write("histogram", osm_hist.to_dict())

# Save layer pyramid
for zoom, layer in sorted(osm_pyramid.levels.items(), reverse=True):
    print("Writing zoom", zoom)
    store.layer(osm_layer_name, zoom).delete("metadata")
    gps.write(layer_uri, osm_layer_name, layer)

## Displaying the Rasterized OSM Features

In [ ]:
osm_layer_name = "rasterized-osm-features"
osm_hist = gps.Histogram.from_dict(store.layer(osm_layer_name).read("histogram"))
osm_color_map = gps.ColorMap.build(osm_hist, 'magma')

osm_tms = gps.TMS.build((layer_uri, osm_layer_name), osm_color_map)
osm_tms.bind("0.0.0.0", 56583)

In [ ]:
osm_map = folium.Map()
folium.TileLayer(tiles='http://localhost:56583/tile/{z}/{x}/{y}.png',
                 attr="GeoPySpark").add_to(osm_map)

In [ ]:
osm_map

## Reading and Formatting the NLCD Data

In [ ]:
# Reading NLCD Data
nlcd = gps.geotiff.get(
    gps.LayerType.SPATIAL, 
    "s3://gt-rasters/nlcd/2011/tiles", 
    crs="epsg:4326", 
    max_tile_size=512, 
    num_partitions=1000)

In [ ]:
tiled_nlcd = nlcd.tile_to_layout(
    osm_raster.layer_metadata, 
    target_crs=4326, 
    partition_strategy=gps.SpatialPartitionStrategy(1000))

In [ ]:
# Reclassify the NLCD values based on estimated walking impact

nlcd_map = {
    11.0: 0.0,
    12.0: 0.15,
    21.0: 0.9,
    22.0: 0.9,
    23.0: 0.9,
    24.0: 0.95,
    31.0: 0.1,
    41.0: 0.7,
    42.0: 0.65,
    43.0: 0.75,
    51.0: 0.75,
    52.0: 0.75,
    71.0: 0.8,
    81.0: 0.8,
    82.0: 0.8,
    90.0: 0.2,
    95.0: 0.25
}

nlcd_pmts = tiled_nlcd\
    .convert_data_type(gps.CellType.FLOAT32, 0.0)\
    .reclassify(nlcd_map, float, gps.ClassificationStrategy.EXACT)

## Saving the Reclassified NLCD Layer

In [ ]:
nlcd_wm = nlcd_pmts.tile_to_layout(gps.GlobalLayout(), target_crs=3857)
nlcd_pyramid = nlcd_wm.pyramid(partition_strategy=gps.SpatialPartitionStrategy(1000))

nlcd_layer_name = "raclassified-nlcd"

# Save layer histogram for later use
nlcd_hist = nlcd_pyramid.get_histogram()
store.layer(nlcd_layer_name).write("histogram", nlcd_hist.to_dict())

# Save layer pyramid
for zoom, layer in sorted(nlcd_pyramid.levels.items(), reverse=True):
    print("Writing zoom", zoom)
    store.layer(nlcd_layer_name, zoom).delete("metadata")
    gps.write(layer_uri, nlcd_layer_name, layer)

## Displaying the Reclassified NLCD Data

In [ ]:
nlcd_layer_name = "raclassified-nlcd"
nlcd_hist = gps.Histogram.from_dict(store.layer(nlcd_layer_name).read("histogram"))
nlcd_color_map = gps.ColorMap.build(nlcd_hist, 'magma')

nlcd_tms = gps.TMS.build((layer_uri, nlcd_layer_name), nlcd_color_map)
nlcd_tms.bind("0.0.0.0", 54970)

In [ ]:
nlcd_map = folium.Map()
folium.TileLayer(tiles='http://localhost:54970/tile/{z}/{x}/{y}.png',
                 attr="GeoPySpark").add_to(nlcd_map)

In [ ]:
nlcd_map

## Reading and Formatting the NED Data

In [ ]:
ned_location = 's3://azavea-datahub/raw/ned-13arcsec-geotiff/'

In [ ]:
ned = gps.geotiff.get(
    gps.LayerType.SPATIAL, 
    ned_location, 
    num_partitions=1000, 
    max_tile_size=256)

In [ ]:
tiled_ned = ned.tile_to_layout(
    osm_raster.layer_metadata, 
    partition_strategy=gps.SpatialPartitionStrategy(1000)
).convert_data_type(gps.CellType.FLOAT32, 0.0).persist(StorageLevel.MEMORY_AND_DISK_SER)

## Saving the NED Layer

In [ ]:
ned_wm = tiled_ned.tile_to_layout(gps.GlobalLayout(), target_crs=3857)
ned_pyramid = ned_wm.pyramid(partition_strategy=gps.SpatialPartitionStrategy(1000))

ned_layer_name = "ned-layer"

# Save layer histogram for later use
ned_hist = ned_pyramid.get_histogram()
store.layer(ned_layer_name).write("histogram", ned_hist.to_dict())

# Save layer pyramid
for zoom, layer in sorted(ned_pyramid.levels.items(), reverse=True):
    print("Writing zoom", zoom)
    store.layer(ned_layer_name, zoom).delete("metadata")
    gps.write(layer_uri, ned_layer_name, layer)

## Displaying the NED Data

In [ ]:
ned_layer_name = "ned-layer"
ned_hist = gps.Histogram.from_dict(store.layer(ned_layer_name).read("histogram"))
ned_color_map = gps.ColorMap.build(ned_hist, 'magma')

ned_tms = gps.TMS.build((layer_uri, ned_layer_name), ned_color_map)
ned_tms.bind("0.0.0.0", 59610)

In [ ]:
ned_map = folium.Map()
folium.TileLayer(tiles='http://localhost:59610/tile/{z}/{x}/{y}.png',
                 attr="GeoPySpark").add_to(ned_map)

In [ ]:
ned_map

## Caculating Tobler Walking Speeds

In [ ]:
# Calculate Slope from the NED layer
zfactor = gps.geotrellis.zfactor_lat_lng_calculator('Meters')
slope_raster = tiled_ned.slope(zfactor)

In [ ]:
# From the Slope layer, calculate the Tobler walking speed
tobler_raster = slope_raster.tobler()

# Add the Tobler and Reclassified NLCD layers to adjusted the Tobler values
adjusted_tobler = tobler_raster + nlcd_pmts

In [ ]:
# The Friction layer is produced by taking the local max between the adjusted Tobler values and the
# rasterized OSM layer
friction_with_roads = adjusted_tobler.local_max(osm_raster)

In [ ]:
reprojected = friction_with_roads.tile_to_layout(
    target_crs = 3857,
    layout = gps.GlobalLayout(tile_size=256),
    resample_method = gps.ResampleMethod.MAX
).convert_data_type(gps.CellType.FLOAT32, 0.0)

In [ ]:
pyramid = reprojected\
    .pyramid(partition_strategy=gps.SpatialPartitionStrategy(1000))\
    .persist()

## Writing the Friction Layer to S3

In [ ]:
layer_name = "us-friction-surface-layer-tms"

# Save layer histogram for later use
hist = pyramid.get_histogram()
store.layer(layer_name).write("histogram", hist.to_dict())

# Save layer pyramid
for zoom, layer in sorted(pyramid.levels.items(), reverse=True):
    print("Writing zoom", zoom)
    store.layer(layer_name, zoom).delete("metadata")
    gps.write(layer_uri, layer_name, layer)

## Displaying the Fricition Layer

In [ ]:
layer_name = "us-friction-surface-layer-tms"
hist = gps.Histogram.from_dict(store.layer(layer_name).read("histogram"))
color_map = gps.ColorMap.build(hist, 'magma')

tms = gps.TMS.build((layer_uri, layer_name), color_map)
tms.bind("0.0.0.0", 55110)

In [ ]:
friction_map = folium.Map()
folium.TileLayer(tiles='http://localhost:55110/tile/{z}/{x}/{y}.png',
                 attr="GeoPySpark").add_to(friction_map)

In [ ]:
friction_map

# Calculating Cost Distance From the Friction Layer

In [ ]:
# The point of interest
point = Point(-75.15415012836456, 39.96134940667086)

In [ ]:
# The point of interest needs to be reprojected to WebMercator in order
# to perform cost distance

project = partial(
    pyproj.transform,  
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

reprojected_point = transform(project, point)

In [ ]:
# Calculate Cost Distance using the Quotient of the average walking speed divided by the reprojected
# friction layer
cost_distance = gps.cost_distance(3.74 / reprojected,
                                  [reprojected_point],
                                  max_distance=50000).persist(StorageLevel.MEMORY_AND_DISK_SER)

## Saving the Cost Distance Layer

In [ ]:
cost_pyramid = cost_distance.pyramid(partition_strategy=gps.SpatialPartitionStrategy(1000))

cost_layer_name = "cost-distance-2"

# Save layer histogram for later use
cost_hist = cost_pyramid.get_histogram()
store.layer(cost_layer_name).write("histogram", cost_hist.to_dict())

# Save layer pyramid
for zoom, layer in sorted(cost_pyramid.levels.items(), reverse=True):
    print("Writing zoom", zoom)
    store.layer(cost_layer_name, zoom).delete("metadata")
    gps.write(layer_uri, cost_layer_name, layer)

## Displaying the Cost Distance Layer

In [ ]:
cost_layer_name = "cost-distance-2"
cost_hist = gps.Histogram.from_dict(store.layer(cost_layer_name).read("histogram"))
cost_color_map = gps.ColorMap.build(cost_hist, 'magma')
cost_tms = gps.TMS.build((layer_uri, cost_layer_name), cost_color_map)
cost_tms.bind("0.0.0.0", 50208)

In [ ]:
cost_map = folium.Map()
folium.TileLayer(tiles='http://localhost:50208/tile/{z}/{x}/{y}.png',
                 attr="GeoPySpark").add_to(cost_map)

In [ ]:
cost_map

![title](https://user-images.githubusercontent.com/11041248/39698721-46d6c59a-51f6-11e8-8e8a-847157280110.png)

![title](https://user-images.githubusercontent.com/11041248/39698722-46ff4d12-51f6-11e8-8e38-b4a6f1bf45d6.png)

![title](https://user-images.githubusercontent.com/11041248/39698723-47286404-51f6-11e8-9873-39eab072060b.png)

![title](https://user-images.githubusercontent.com/11041248/39698724-474dc960-51f6-11e8-9e9a-714137a78d8d.png)

![title](https://user-images.githubusercontent.com/11041248/39698725-477b7a54-51f6-11e8-9188-319d951ecea4.png)

![title](https://user-images.githubusercontent.com/11041248/39698775-855479f2-51f6-11e8-9bd3-307bf0a4dabd.png)

![title](https://user-images.githubusercontent.com/11041248/39698776-85804bae-51f6-11e8-8dbe-758df3276a13.png)

![title](https://user-images.githubusercontent.com/11041248/39698777-85a95026-51f6-11e8-8542-61fdbad39614.png)

![title](https://user-images.githubusercontent.com/11041248/39806706-d38d70d0-537a-11e8-977a-fc06ca577a57.png)